> 2번 문제 풀면서 만들어진 df가 3번 문제 풀 때 영향을 미칩니다

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
df_origin = pd.read_csv("../Datasets/edu_enrollees.csv")

삼성전자 임직원의 체계적인 커리어 패스 설계를 위해 데이터 분석과 같은 Hard Skill 역량과 Soft Skill 역량을 조화롭게 함양하기 위해 HRD 부서의 조프로는 전산팀의 협조를 받아 데이터를 확보하였다.  
  
※ 분석을 수행하기 전, 상기 데이터를 이용하여 아래의 전처리를 수행하시오.  
  
* 단계 1: 분석에 사용하지 않을 city, company_size, company_type 컬럼을 제거하시오.  
* 단계 2: 각 문자형 변수에 결측치(null/empty space)가 하나라도 존재하는 행은 모두 제거하시오.  
* 단계 3: experience 변수 값이 ‘>20’ 또는 ‘<1’인 값을 제거하고 experience 컬럼의 타입을 정수형으로 변환하시오.  
* 단계 4: last_new_job 변수의 값이 ‘>4’ 또는 ‘never’인 값을 제거하고 해당변수의 타입을 정수형으로 변환하시오.  
  
전처리 수행 이후 행 개수는 7,522이며 전처리가 완료된 데이터를 base 객체로 지정하고 이를 사용하여 문제를 풀이하시오.  
  

* 단계 1: 분석에 사용하지 않을 city, company_size, company_type 컬럼을 제거하시오.  

In [30]:
df_origin = df_origin.drop(["city", "company_size", "company_type"], axis = 1)

* 단계 2: 각 문자형 변수에 결측치(null/empty space)가 하나라도 존재하는 행은 모두 제거하시오.  

In [31]:
df_origin = df_origin.dropna()

> empty space가 있을 때가 있다!

In [32]:
df_origin.apply(lambda x: x=='').sum(axis=0)

enrollee_id               0
city_development_index    0
gender                    0
relevant_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
last_new_job              0
training_hours            0
target                    0
Xgrp                      0
dtype: int64

In [33]:
(df_origin=='').sum()

enrollee_id               0
city_development_index    0
gender                    0
relevant_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
last_new_job              0
training_hours            0
target                    0
Xgrp                      0
dtype: int64

* 단계 3: experience 변수 값이 ‘>20’ 또는 ‘<1’인 값을 제거하고 experience 컬럼의 타입을 정수형으로 변환하시오.  

In [34]:
df_origin = df_origin.loc[~(df_origin["experience"].isin([">20", "<1"])), ]

In [35]:
df_origin['experience']=df_origin['experience'].astype('int')

* 단계 4: last_new_job 변수의 값이 ‘>4’ 또는 ‘never’인 값을 제거하고 해당변수의 타입을 정수형으로 변환하시오.  

In [36]:
df_origin = df_origin.loc[~df_origin["last_new_job"].isin([">4", "never"]), ]
df_origin["last_new_job"] = df_origin["last_new_job"].astype("int")

  
전처리 수행 이후 행 개수는 7,522이며 전처리가 완료된 데이터를 base 객체로 지정하고 이를 사용하여 문제를 풀이하시오.  

In [37]:
len(df_origin)

7522

#### Q1. (base를 사용하여)관련 분야 경험 여부(relevant_experience)에 따른 전배 희망 여부(target)를 기술통계량으로 확인하고자 한다. 관련 분야 경험이 없는 수료자 중 전배를 희망하는 수료자의 비율을 A, 관련 분야 경험이 있는 수료자 중 전배를 희망하는 수료자의 비율을 B라 할 때, A/B를 구하시오.
※ 관련 경험이 없는 사람은 relevant_experience변수의 값이 ‘No relevant experience’인 사람으로 정의한다.  
※ 관련 경험이 있는 사람은 relevant_experience변수의 값이 ‘Has relevant experience’인 사람으로 정의한다.  
※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [38]:
df = df_origin.copy()

In [39]:
df["relevant_experience"].unique()

array(['Has relevant experience', 'No relevant experience'], dtype=object)

In [40]:
df["target"].unique()

array([1., 0.])

In [41]:
df_no  = df.loc[df["relevant_experience"] == "No relevant experience"]
df_has = df.loc[df["relevant_experience"] == "Has relevant experience"]

In [42]:
df_no["target"].value_counts()

0.0    872
1.0    541
Name: target, dtype: int64

In [43]:
df_no["target"].value_counts(normalize = True)

0.0    0.617127
1.0    0.382873
Name: target, dtype: float64

> 비율을 구하는 거니깐 이렇게 풀면 된다

In [44]:
stat_no = df_no["target"].value_counts(normalize = True)[1]
stat_no

0.38287331917905165

In [45]:
stat_has = df_has["target"].value_counts(normalize = True)[1]
stat_has

0.21591095105581928

In [46]:
round(stat_no / stat_has, 2)

1.77

#### Q2. (base를 사용하여) 전배 희망 여부(target)에 영향을 주는 변수들을 확인하고자 한다. 다음 절차에 따라 로지스틱 회귀분석을 수행하고 질문에 답하시오.
* 단계 1: gender, relevant_experience, enrolled_university, education_level, major_discipline 변수로부터 더미 변수들을 생성한다. 단, 각 변수로부터 더미 변수를 생성할 때 마지막으로 등장하는 범주는 제외하도록 한다. (여기서 마지막으로 등장하는 범주란, 각 컬럼의 값을 사전 순으로 나열하였을 때 마지막으로 등장하는 값이다. 예를 들어, ‘col’ 변수의 범주가 \[‘A’,’C’,’B’,’B’,’C’,’A’\]의 값을 가진다면 사전 상의 마지막 값인 C가 제외된다)
* 단계 2: 단계 1에서 생성한 더미 변수와 city_development_index, experience, last_new_job, training_hours, target, Xgrp 변수를 결합하여 새로운 데이터셋(데이터셋명: job2, 이 데이터셋은 문제 3에서도 활용)을 구성한다. 이 때, target, Xgrp를 제외한 데이터셋의 컬럼은 아래 순서에 따르도록 한다.  
\- city_development_index  
\- experience  
\- last_new_job  
\- training_hours  
\- gender의 더미 변수  
\- relevant_experience의 더미 변수  
\- enrolled_university의 더미 변수  
\- education_level의 더미 변수  
\- major_discipline의 더미 변수  

* 단계 3. 단계 2에서 구성한 데이터셋 job2로 다음 조건에 따라 상수항(Intercept)이 포함된 로지스틱 회귀분석을 수행한다.  
\- 종속 변수 : target  
\- 독립 변수(총 16개) : target과 Xgrp를 제외한 나머지 변수  
\- 회귀식에 포함되는 독립 변수의 순서를 컬럼의 순서와 일치시킨다.  

상수항을 제외한 나머지 변수들에 대한 Odds Ratio중 가장 큰 값을 기술하시오.  

$$ x_i 의\: Odds Ratio = \frac{odds(P(Y=1|x_1,\: \dots, \: x_i + 1,\: \dots ,\: x_n))}{odds(P(Y=1|x_1,\: \dots ,\:  x_i,\: \dots ,\: x_n)} $$

※ LogisticRegression() 클래스의 인자 C는 100000으로 지정하시오.  
※ LogisticRegression() 클래스의 인자 random_state는 123으로 지정하시오.  
※ 정답은 소수점 셋째 자리에서 버림하여 둘째 자리까지 출력하시오.  
(정답 예시: 0.123)

---

In [47]:
df=df_origin.copy()

※ LogisticRegression() 클래스의 인자 C는 100000, solver='liblinear', max_iter=1000  으로 지정하시오.  
※ LogisticRegression() 클래스의 인자 random_state는 123으로 지정하시오.  

### params

In [48]:
C = 100_000
random_state = 123
solver='liblinear'
max_iter=1000

* 단계 1: gender, relevant_experience, enrolled_university, education_level, major_discipline 변수로부터 더미 변수들을 생성한다. 단, 각 변수로부터 더미 변수를 생성할 때 마지막으로 등장하는 범주는 제외하도록 한다. (여기서 마지막으로 등장하는 범주란, 각 컬럼의 값을 사전 순으로 나열하였을 때 마지막으로 등장하는 값이다. 예를 들어, ‘col’ 변수의 범주가 \[‘A’,’C’,’B’,’B’,’C’,’A’\]의 값을 가진다면 사전 상의 마지막 값인 C가 제외된다)

In [49]:
df_t = pd.DataFrame(dict(aa = ["a", "A", "C", "D"]))
pd.get_dummies(df_t)

,aa_A,aa_C,aa_D,aa_a
0,0,0,0,1
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0


> aa_a를 삭제하란 말이네

In [50]:
df_cat = df[["gender", "relevant_experience", "enrolled_university", "education_level", "major_discipline"]]
df_cat_dum = pd.get_dummies(df_cat)

In [51]:
df_cat_dum

,gender_Female,gender_Male,gender_Other,relevant_experience_Has relevant experience,relevant_experience_No relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_Masters,education_level_Phd,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM
8,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1
11,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1
19,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0
20,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1
21,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19149,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1
19150,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1
19152,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0
19153,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0


In [52]:
df_cat_dum = df_cat_dum.drop(["gender_Other", "relevant_experience_No relevant experience", 
                              "enrolled_university_no_enrollment", "education_level_Phd",
                              "major_discipline_STEM"],
                             axis = 1)
df_cat_dum.head(1)

,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
8,0,1,1,0,0,1,0,0,0,0,0,0


>  각 범주의 unique 구해서 하나씩 저장해서 컬럼에서 빼도 되겠지만 몇 개 안되니깐 빨리 풀고 싶으면 직접 입력

In [53]:
col_cat=df_cat.columns

In [54]:
col_drop = []
for cat in col_cat:
    print(cat)
    drop_cal =cat+'_'+df_cat[cat].unique()[-1]
    col_drop = col_drop + [drop_cal]

gender
relevant_experience
enrolled_university
education_level
major_discipline


In [55]:
col_drop

['gender_Other',
 'relevant_experience_No relevant experience',
 'enrolled_university_Part time course',
 'education_level_Phd',
 'major_discipline_No Major']

In [57]:
# df_cat_dum.drop(columns=col_drop)

* 단계 2: 단계 1에서 생성한 더미 변수와 city_development_index, experience, last_new_job, training_hours, target, Xgrp 변수를 결합하여 새로운 데이터셋(데이터셋명: job2, 이 데이터셋은 문제 3에서도 활용)을 구성한다. 이 때, target, Xgrp를 제외한 데이터셋의 컬럼은 아래 순서에 따르도록 한다.  
\- city_development_index  
\- experience  
\- last_new_job  
\- training_hours  
\- gender의 더미 변수  
\- relevant_experience의 더미 변수  
\- enrolled_university의 더미 변수  
\- education_level의 더미 변수  
\- major_discipline의 더미 변수  

In [58]:
col_select = ["training_hours","experience","last_new_job","city_development_index", "target", "Xgrp"]
df_job2 = pd.concat([df[col_select],
                     df_cat_dum],
                    axis = 1)

* 단계 3. 단계 2에서 구성한 데이터셋 job2로 다음 조건에 따라 상수항(Intercept)이 포함된 로지스틱 회귀분석을 수행한다.  
\- 종속 변수 : target  
\- 독립 변수(총 16개) : target과 Xgrp를 제외한 나머지 변수  
\- 회귀식에 포함되는 독립 변수의 순서를 컬럼의 순서와 일치시킨다.  

상수항을 제외한 나머지 변수들에 대한 Odds Ratio중 가장 큰 값을 기술하시오.  

$$ x_i 의\: Odds Ratio = \frac{odds(P(Y=1|x_1,\: \dots, \: x_i + 1,\: \dots ,\: x_n))}{odds(P(Y=1|x_1,\: \dots ,\:  x_i,\: \dots ,\: x_n)} $$


※ 정답은 소수점 셋째 자리에서 버림하여 둘째 자리까지 출력하시오.  
(정답 예시: 0.123)

In [59]:
df_job2.shape

(7522, 18)

In [60]:
df_job2.head(1)

,training_hours,experience,last_new_job,city_development_index,target,Xgrp,gender_Female,gender_Male,relevant_experience_Has relevant experience,enrolled_university_Full time course,enrolled_university_Part time course,education_level_Graduate,education_level_Masters,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other
8,46.0,7,1,0.92,1.0,train,0,1,1,0,0,1,0,0,0,0,0,0


In [61]:
model_lr = LogisticRegression(C=C, random_state=random_state, max_iter=max_iter, solver=solver)

In [62]:
X = df_job2.drop('Xgrp', axis=1)

In [63]:
y = X.pop('target')

In [64]:
model_lr.fit(X =X, y = y)

LogisticRegression(C=100000, max_iter=1000, random_state=123,
                   solver='liblinear')

In [65]:
model_lr.coef_

array([[-9.41337371e-04, -2.86714152e-02,  9.41239768e-02,
        -6.16992454e+00, -2.14110739e-01, -1.83586847e-01,
        -7.68926670e-01,  5.13463387e-01, -2.90034142e-01,
         2.25348707e-01, -7.57908730e-02,  2.84870399e-01,
         1.17049893e-01,  2.45877610e-01,  4.02026554e-01,
        -4.42105987e-01]])

In [66]:
model_lr.intercept_

array([4.55618151])

In [67]:
import numpy as np

In [68]:
round(np.exp(model_lr.coef_).round(3).max(),3).astype(str)[:-1]

'1.67'

### Q3. (job2를 이용하여) 전체 데이터를 Train과 Test Set으로 나누고, Train Set으로 학습한 모델을 Test Set에 적용하여 모델을 평가하고자 한다. 다음 절차에 따라 분석을 수행하고 질문에 답하시오.

* 단계 1: 2단계에서 구성한 데이터셋 job2 에서 Xgrp 컬럼의 값이 ‘train’인 경우 Train Set으로,‘test’인 경우 Test Set으로 정의하여 분할한다.
* 단계 2: 아래 가이드에 따라 Train Set으로 K-NN 분류 모델을 학습하고, 이 모델을 Test Set에 적용한다.  
\- 종속 변수: 전배 희망 여부(target)  
\- 독립 변수(총 16개): 전배 희망 여부(target)와 Train/Test set 구분 변수(Xgrp)를 제외한 모든 변수  
\- Euclidean 거리 기준 가장 가까운 5개 데이터의 ‘전배 희망 여부(target)’를 활용하여 예측  
* 단계 3: 예측 결과를 바탕으로 아래 정의된 지표 A를 계산하여 기술하시오.

$$ 𝐴=((\#\: of\: true\: positive)+(\#\: of\: true\: negative))/((\#\: of\: total\: data)) $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

* 단계 1: 2단계에서 구성한 데이터셋 job2 에서 Xgrp 컬럼의 값이 ‘train’인 경우 Train Set으로,‘test’인 경우 Test Set으로 정의하여 분할한다.


In [69]:
df_job2["Xgrp"].unique()

array(['train', 'test'], dtype=object)

In [70]:
df_train = df_job2.loc[df_job2["Xgrp"] == "train", ]
df_test  = df_job2.loc[df_job2["Xgrp"] == "test" , ]

* 단계 2: 아래 가이드에 따라 Train Set으로 K-NN 분류 모델을 학습하고, 이 모델을 Test Set에 적용한다.  
\- 종속 변수: 전배 희망 여부(target)  
\- 독립 변수(총 16개): 전배 희망 여부(target)와 Train/Test set 구분 변수(Xgrp)를 제외한 모든 변수  
\- Euclidean 거리 기준 가장 가까운 5개 데이터의 ‘전배 희망 여부(target)’를 활용하여 예측  


In [71]:
df_train=df_train.drop(columns='Xgrp')
df_test=df_test.drop(columns='Xgrp')

In [72]:
y_train=df_train.pop('target')
y_test = df_test.pop('target')

In [73]:
X_train = df_train.copy()
X_test = df_test.copy()

In [74]:
del df_train, df_test

In [75]:
X_train.columns.shape

(16,)

In [76]:
model = KNeighborsClassifier(n_neighbors = 5,   p=2,     metric='minkowski') #유클리디안 디폴트
model.fit(X = X_train,
          y = y_train)
y_pred = model.predict(X_test)

In [77]:
np.unique(y_pred)

array([0., 1.])

In [78]:
np.unique(y_pred,return_counts=True)

(array([0., 1.]), array([2514,  302], dtype=int64))

* 단계 3: 예측 결과를 바탕으로 아래 정의된 지표 A를 계산하여 기술하시오.

$$ 𝐴=((\#\: of\: true\: positive)+(\#\: of\: true\: negative))/((\#\: of\: total\: data)) $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [79]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1900,  192],
       [ 614,  110]], dtype=int64)

In [80]:
total_data=confusion_matrix(y_test, y_pred).sum()

In [81]:
tn=confusion_matrix(y_test, y_pred)[0,0]
tp=confusion_matrix(y_test, y_pred)[1,1]

In [82]:
round((tn+tp)/total_data, 2)

0.71

In [84]:
from sklearn.metrics import accuracy_score
round(accuracy_score(y_true = y_test, 
                     y_pred = y_pred), 2)

0.71

# ====추가 문제====

In [102]:
df4=df_origin.copy()

Q. training_hours의 왜도와 첨도를 더한 값은?

In [103]:
df4.training_hours.skew()+df4.training_hours.kurtosis()

5.765837001850359

Q. pd.qcut을 이용하여 training_hours를 4등분 하라.   
그리고 가장 작은 그룹에서 세번째 그룹까지의 training_hours 중앙값을 더하시오

In [104]:
pd.qcut(df4.training_hours, q=4)

8         (23.0, 48.0]
11       (88.0, 336.0]
19       (88.0, 336.0]
20        (48.0, 88.0]
21        (48.0, 88.0]
             ...      
19149     (23.0, 48.0]
19150    (0.999, 23.0]
19152     (23.0, 48.0]
19153     (23.0, 48.0]
19154     (48.0, 88.0]
Name: training_hours, Length: 7522, dtype: category
Categories (4, interval[float64, right]): [(0.999, 23.0] < (23.0, 48.0] < (48.0, 88.0] < (88.0, 336.0]]

In [105]:
# 구간 분할
df4['range']=pd.qcut(df4.training_hours, q=4, labels=['group1','group2','group3','group4'])

In [113]:
df4['range'].value_counts(normalize=True)

group1    0.256847
group2    0.253523
group4    0.249535
group3    0.240096
Name: range, dtype: float64

In [ ]:
df4['range'].groupby

In [121]:
df4.groupby('range')['training_hours'].median()

range
group1     13.0
group2     35.0
group3     64.0
group4    132.0
Name: training_hours, dtype: float64

In [127]:
df4.groupby('range')['training_hours'].median().cumsum()

range
group1     13.0
group2     48.0
group3    112.0
group4    244.0
Name: training_hours, dtype: float64

In [129]:
df4.groupby('range')['training_hours'].median().cumsum()[-2]

112.0

In [135]:
df4

,enrollee_id,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,last_new_job,training_hours,target,Xgrp,range
8,27107.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,1,46.0,1.0,train,group2
11,23853.0,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,5,1,108.0,0.0,train,group4
19,11399.0,0.827,Female,Has relevant experience,no_enrollment,Graduate,Arts,4,1,132.0,1.0,train,group4
20,31972.0,0.843,Male,Has relevant experience,no_enrollment,Masters,STEM,11,1,68.0,0.0,train,group3
21,19061.0,0.926,Male,Has relevant experience,no_enrollment,Masters,STEM,11,2,50.0,0.0,train,group3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19149,251.0,0.920,Male,Has relevant experience,no_enrollment,Masters,STEM,9,1,36.0,1.0,test,group2
19150,32313.0,0.920,Female,Has relevant experience,no_enrollment,Graduate,STEM,10,3,23.0,0.0,test,group1
19152,29754.0,0.920,Female,Has relevant experience,no_enrollment,Graduate,Humanities,7,1,25.0,0.0,test,group2
19153,7386.0,0.878,Male,No relevant experience,no_enrollment,Graduate,Humanities,14,1,42.0,1.0,test,group2


Q. 인사팀 내부에서 관리하는 training_hours2 파생변수를 생성하려고 한다.   
(생성규칙) 만약 관련 경험이 있으면 training_hours에 1.2를 곱하고 그렇지 않으면 0.8을 곱하라

In [162]:
def weight_hours(df):
    if df['relevant_experience'] == 'Has relevant experience':
        df['training_hours2']= df['training_hours']*1.2
    else:
        df['training_hours2']= df['training_hours']*.8
    return df

In [171]:
df4.apply(weight_hours, axis=1)[['training_hours','relevant_experience','training_hours2']].tail()

,training_hours,relevant_experience,training_hours2
19149,36.0,Has relevant experience,43.2
19150,23.0,Has relevant experience,27.6
19152,25.0,Has relevant experience,30.0
19153,42.0,No relevant experience,33.6
19154,52.0,Has relevant experience,62.4
